In [ ]:
pip install -qU langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

데이터

In [2]:
import json

file_path = 'data.json'  

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [3]:
import ipdb

def extract_spending_details_by_category(data):
    categories = ['본식', '스드메', '예복', '예물/예단', '신혼여행', '가전/가구', '기타']
    
    result = {category: [] for category in categories}
    
    for category_data in data['groupedSpendings']:
        category_name = category_data['budgetCategory']['budgetCategoryName']
        # totalAmount은 사용 x
        #ipdb.set_trace()
        
        if category_name in categories:
            for spending in category_data['spendings']:
                spending_details = {
                    f"{spending['title']}": spending['amount']
                }
                result[category_name].append(spending_details)
    
    return result


extracted_data = extract_spending_details_by_category(data)

print(extracted_data)

{'본식': [{'식대': 0}, {'홀 대관료': 1000000}], '스드메': [{'스드메': 1439000}], '예복': [{'예복': 0}], '예물/예단': [{'결혼반지': 2390000}], '신혼여행': [{'숙박비': 0}, {'항공권': 0}, {'투어 및 기타 경비': 0}], '가전/가구': [{'청소기': 759000}, {'침대': 8102000}, {'삼성': 8770000}, {'식탁': 722000}, {'쇼파': 3204820}], '기타': [{'줄눈': 450000}, {'탄성코트': 550000}, {'하자점검': 270000}, {'입주청소': 510000}, {'TV(무타공)': 330000}]}


프롬프트

In [4]:
# Simple version
PROMPT_TEMPLATE_EG2 = """
# Instruction 
You are an AI assistant helping with wedding budget management. Please respond to the user's questions kindly and accurately. Your goal is to provide helpful advice based on the user’s current wedding budget, suggest possible savings or cost-efficient alternatives, and, when needed, provide additional information by using available tools to search the internet for price comparisons, recommendations, or tips. Whenever possible, actively leverage available tools to provide the most accurate and up-to-date information to the user.

Current wedding budget information:
{budget_info}

TOOLS:
----

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```

Thought: Do I need to use a tool? Yes

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```

Thought: Do I need to use a tool? No

Final Answer: [your response here]

```

Begin!

Previous conversation history:

{chat_history}

New input: {input}

{agent_scratchpad}

"""

In [5]:
# English 
PROMPT_TEMPLATE_EG = """
# Instruction 
You are an AI assistant helping with wedding budget management. Please respond to the user's questions kindly and accurately. Your goal is to provide helpful advice based on the user’s current wedding budget, suggest possible savings or cost-efficient alternatives, and, when needed, provide additional information by using available tools to search the internet for price comparisons, recommendations, or tips. Whenever possible, actively leverage available tools to provide the most accurate and up-to-date information to the user.

Current wedding budget information:
{budget_info}

# Tools 
- Assistant has access to the following tools:

{tools}

- To use a tool, please use the following format:

Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, MUST be written in Korean and be related to wedding
Observation: the result of the action
...the Thought/Action/Action Input/Observation cannot repeat several times

- When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

Thought: No
Final Answer: [your response here]

# GUIDELINES:

1. Actively seek opportunities to use available tools to provide the user with the most relevant and up-to-date information (e.g., price comparisons for specific wedding expenses or vendor recommendations).
2. Always consider the user’s wedding budget limitations and provide cost-efficient suggestions where possible.
3. When there are zero values or missing information in certain categories, offer advice on possible cost ranges or alternatives based on typical wedding expenditures.
4. For large expenses, help the user identify areas where they might save by comparing current prices online or recommending alternative options (e.g., for furniture or honeymoon).
5. If there are optional categories, ask the user if they would like to explore further options for those categories (e.g., entertainment, videography).
6. Only respond to queries that are directly related to wedding budget management or planning.
7. Keep your tone friendly, empathetic, and professional.

# IMPORTANT : 
If you have found the answer or if you cannot find the answer after using tools, you MUST respond with a Final Answer.

# Begin!

Previous conversation history:
{chat_history}

New input: {input}

{agent_scratchpad}

"""

In [6]:
# Korean
PROMPT_TEMPLATE = """
# INSTRUCTION
당신은 결혼 예산 관리를 돕는 AI 어시스턴트입니다. 사용자의 질문에 친절하고 정확하게 답변하세요. 목표는 사용자의 현재 결혼 예산을 기반으로 도움이 되는 조언을 제공하고, 가능한 절약 방법이나 비용 효율적인 대안을 제안하며, 필요할 경우 인터넷 검색 도구를 사용해 가격 비교, 추천, 팁 등의 추가 정보를 제공하는 것입니다. 가능한 한 적극적으로 도구를 활용하여 사용자에게 가장 정확하고 최신의 정보를 한국어로 제공하세요.

현재 결혼 예산 정보:
{budget_info}

# TOOLS 
AI 어시스턴트는 다음 도구들에 접근할 수 있습니다:

{tools}

----

도구를 사용하려면 다음 형식을 사용하세요:

생각: 도구를 사용할 필요가 있는가? 예
액션: [{tool_names}] 중 하나를 선택해 실행할 작업
액션 입력: 작업에 필요한 입력
관찰: 작업 실행 결과

사용자에게 응답할 때, 또는 도구를 사용할 필요가 없을 때는 반드시 다음 형식을 사용하세요:

생각: 아니요
최종 답변: [여기에 당신의 응답을 입력하세요]

----

# 가이드라인:

1. 사용 가능한 도구를 적극적으로 활용하여 사용자에게 가장 관련성 있고 최신 정보를 제공하세요 (예: 특정 결혼 비용에 대한 가격 비교 또는 업체 추천).
2. 항상 사용자의 결혼 예산 한도를 고려하고, 가능한 비용 효율적인 제안을 제공하세요.
3. 특정 항목에서 값이 없거나 정보가 누락된 경우, 일반적인 결혼 비용을 기준으로 가능한 비용 범위나 대안을 제시하세요.
4. 큰 지출 항목에 대해서는 온라인에서 현재 가격을 비교하거나 대체 옵션을 추천하여 절약할 수 있는 부분을 도와주세요 (예: 가구나 신혼여행).
5. 선택 가능한 항목이 있다면, 해당 항목에 대해 더 많은 옵션을 탐색할지 사용자에게 물어보세요 (예: 엔터테인먼트, 비디오 촬영).
6. 결혼 예산 관리나 계획과 관련된 질문에만 답변하세요.
7. 친절하고, 공감적이며, 전문가다운 톤을 유지하세요.

# 시작!

이전 대화 기록:
{chat_history}

새로운 입력: {input}

{agent_scratchpad}

"""


In [70]:
import os
import readline
import json
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from langchain.tools.render import render_text_description
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.prompts import PromptTemplate


def custom_input(prompt):
    readline.set_pre_input_hook(lambda: readline.insert_text(""))
    readline.set_startup_hook(lambda: readline.insert_text(""))
    try:
        return input(prompt)
    finally:
        readline.set_pre_input_hook(None)
        readline.set_startup_hook(None)


def main():

    memory = ConversationBufferMemory(memory_key="chat_history", input_key="input") #  #

    tools = [TavilySearchResults(max_results=5)]


    prompt = PromptTemplate(
        input_variables=["agent_scratchpad", "chat_history", "input", "budget_info", "tools", "tool_names"],
        template=PROMPT_TEMPLATE_EG
        )
    
    prompt = prompt.partial(
        tools=render_text_description(tools),
        tool_names=", ".join([t.name for t in tools]),
    )
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    llm_with_stop = llm.bind(stop=["\nObservation"])

    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
            "chat_history": lambda x: x["chat_history"],
            "budget_info": lambda x: x["budget_info"]
        }
        | prompt
        | llm_with_stop
        | ReActSingleInputOutputParser()
    )
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True,
        handle_parsing_errors=True
    )

    while True:
        user_input = custom_input("당신: ")
        if user_input.lower() == "exit":
            print("어시스턴트: 안녕히 가세요!")
            break

        try:
            response = agent_executor.invoke({
                "input": user_input,
                "budget_info": json.dumps(extracted_data, ensure_ascii=False)
            })
            print("어시스턴트: " + response["output"])

        except Exception as e:
            print(f"어시스턴트: 죄송합니다. 오류가 발생했습니다: {str(e)}")


if __name__ == "__main__":
    main()




> Entering new AgentExecutor chain...
Thought: No
Final Answer: 안녕하세요! 결혼 예산 관리에 대해 도움이 필요하시면 언제든지 말씀해 주세요. 어떤 질문이든 환영합니다!

> Finished chain.
어시스턴트: 안녕하세요! 결혼 예산 관리에 대해 도움이 필요하시면 언제든지 말씀해 주세요. 어떤 질문이든 환영합니다!


> Entering new AgentExecutor chain...
Thought: No
Final Answer: 결혼 준비로 바쁘신 것 같네요! 배고프시다면 간단한 간식이나 식사를 챙기시는 것도 좋을 것 같습니다. 결혼 준비와 관련된 질문이 있으시면 언제든지 말씀해 주세요!

> Finished chain.
어시스턴트: 결혼 준비로 바쁘신 것 같네요! 배고프시다면 간단한 간식이나 식사를 챙기시는 것도 좋을 것 같습니다. 결혼 준비와 관련된 질문이 있으시면 언제든지 말씀해 주세요!


> Entering new AgentExecutor chain...
Thought: No
Final Answer: 현재 제공된 예산 정보를 바탕으로, 전체 결혼 예산은 다음과 같습니다:

1. 본식: 홀 대관료 1,000,000원
2. 스드메: 1,439,000원
3. 예복: 0원
4. 예물/예단: 결혼반지 2,390,000원
5. 신혼여행: 숙박비, 항공권, 투어 및 기타 경비 0원
6. 가전/가구: 총 8,000,000원 이상 (청소기, 침대, 삼성 가전, 식탁, 쇼파 포함)
7. 기타: 총 2,100,000원 (줄눈, 탄성코트, 하자점검, 입주청소, TV 포함)

현재 예산에서 예복과 신혼여행 항목이 0원으로 설정되어 있습니다. 예복과 신혼여행에 대한 예산을 추가하는 것이 좋습니다. 또한, 가전/가구 항목에서 큰 비용이 발생하고 있으므로, 필요한 품목을 재검토하거나 대체 가능한 저렴한 옵션을 고려해 보시는 것도 좋습니다.

더 구체적인 조언이나 추가적인 질문이 있으시면 말씀해 주세요!

> Finish